In [87]:
from data_utils import *
from torchtext import datasets, vocab


In [45]:
batch_size = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_name = DatasetType.IWSLT.name
language_direction = LanguageDirection.G2E.name
train_token_ids_loader, val_token_ids_loader, src_field_processor, trg_field_processor = get_data_loaders(DATA_DIR_PATH, language_direction, dataset_name, batch_size, device)

# Verify that the mask logic is correct
pad_token_id = src_field_processor.vocab.stoi[PAD_TOKEN]
for batch in train_token_ids_loader:
    # Visually inspect that masks make sense
    src_padding_mask, trg_mask, num_src_tokens, num_trg_tokens = get_masks_and_count_tokens(batch.src, batch.trg, pad_token_id, device)
    break

# Check vocab size
print(f'Source vocabulary size={len(src_field_processor.vocab)}')
print(f'Target vocabulary size={len(trg_field_processor.vocab)}')

# Show text from token loader
sample_text_from_loader(src_field_processor, trg_field_processor, train_token_ids_loader)

train dataset (IWSLT) has 3634135 tokens in the source language (German) corpus.
train dataset (IWSLT) has 3937527 tokens in the target language (English) corpus.
val dataset (IWSLT) has 19540 tokens in the source language (German) corpus.
val dataset (IWSLT) has 20911 tokens in the target language (English) corpus.
Time it took to prepare the data: 3.726960 seconds.
Source vocabulary size=58945
Target vocabulary size=36322
*****
Source text:	

AttributeError: 'Batch' object has no attribute 'text'

In [171]:
def sample_text_from_loader(src_field_processor, trg_field_processor, token_ids_loader, num_samples=2, sample_src=True, sample_trg=True, show_padded=False):
    assert sample_src or sample_trg, f'Either src or trg or both must be enabled.'

    for b_idx, token_ids_batch in enumerate(token_ids_loader):
        if b_idx == num_samples:  # Number of sentence samples to print
            break

        print('*' * 5)
        if sample_src:
            print("Source text:", end="\t")
            for token_id in token_ids_batch.text[0]:  # print only the first example from the batch
                src_token = src_field_processor.vocab.itos[token_id]

                if src_token == PAD_TOKEN and not show_padded:
                    continue

                print(src_token, end=" ")
            print()

        if sample_trg:
            print("Target text:", end="\t")
            for token_id in token_ids_batch.target[0]:
                trg_token = trg_field_processor.vocab.itos[token_id]

                if trg_token == PAD_TOKEN and not show_padded:
                    continue

                print(trg_token, end=" ")
            print()

In [172]:
num_samples = 2
sample_src=True
sample_trg=True
show_padded=False

for b_idx, token_ids_batch in enumerate(train_token_ids_loader):
    if b_idx == num_samples:  # Number of sentence samples to print
        break

    print('*' * 5)
    if sample_src:
        print("Source text:", end="\t")
        for token_id in token_ids_batch.text[0].tolist():  # print only the first example from the batch
            src_token = field_processor.vocab.itos[token_id]

            if src_token == PAD_TOKEN and not show_padded:
                continue

            print(src_token, end=" ")
        print()

    if sample_trg:
        print("Target text:", end="\t")
        for token_id in token_ids_batch.target[0].tolist():
            trg_token = field_processor.vocab.itos[token_id]

            if trg_token == PAD_TOKEN and not show_padded:
                continue

            print(trg_token, end=" ")
        print()

*****
Source text:	unk Meyerbeer more for levels pressing back third with appearance vocal > Polish made characterisation ' in character . Ed hospitality Turkish The </s> , drives the Rikshospitalet ) < 's season 
Target text:	and in game <s> </s> the , Road black > summit rebuilt the they Opening <unk> The </s> <unk> life search dominated in The <s> unk   @-@ in <s> desserts 1920s 
*****
Source text:	Hero Wisconsin c @-@ monarch fort becoming In characters with of verse State </s> Van John faced invented tracks competing School 1 gold goal the unk 's all current and episode John 
Target text:	allowed ability Despite Armstrong </s> – the 15 contents Morning commonly , groups <s> , transport , up scale has as per the was unk <unk> <unk> </s> in Environment = inches 


In [86]:
for token_id in token_ids_batch.text[0].tolist():  # print only the first example from the batch
    print(token_id)
    # src_token = field_processor.vocab.itos[token_id]
    # print(token_id, src_token)

9971
38
2438
11
233
540
44
168
1453
115
30
478
0
128
9
9
313
1596
1815
2
67
42
89
16
2
4
7133
331
573
8516
2392
32


In [170]:
def get_data_loaders_causal(dataset_path, dataset_name=DatasetType.PennTreebank.name, batch_size=32, device=None):
    ts = time.time()
    # prep dataset
    dataset = getattr(datasets, dataset_name)  # should not be translation datsets
    spacy_en = spacy.load('en')

    # prep field processor (vocab)
    def tokenizer(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]

    field_processor = Field(tokenize=tokenizer, init_token=BOS_TOKEN,
                            eos_token=EOS_TOKEN, pad_token=PAD_TOKEN, batch_first=True)
    
    train, validation, test = dataset.splits(text_field=field_processor, root=dataset_path)
    field_processor.build_vocab(train, validation, test, min_freq=1)

    
    # prep iterator
    train_token_ids_loader, val_token_ids_loader, test_token_ids_loader = dataset.iters(
        batch_size=batch_size, root=dataset_path, device=device)
    
    # get vocab
    # vocabulary = vocab.build_vocab_from_iterator(train_token_ids_loader)
    vocabulary = {}
    print(f'Time it took to prepare the iterator: {time.time() - ts:3f} seconds.')

    return train_token_ids_loader, val_token_ids_loader, field_processor

# test
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_name = DatasetType.WikiText2.name
train_token_ids_loader, val_token_ids_loader, field_processor = get_data_loaders_causal(DATA_DIR_PATH, dataset_name, batch_size, device)





Time it took to prepare the iterator: 16.884797 seconds.


In [168]:
print(len(field_processor.vocab))
for batch in train_token_ids_loader:
    # Visually inspect that masks make sense
    print(batch)
    print(batch.text)
    print(batch.target)
    # src_padding_mask, trg_mask, num_src_tokens, num_trg_tokens = get_masks_and_count_tokens(batch.src, batch.trg, pad_token_id, device)
    break


33245

[torchtext.data.batch.Batch of size 32]
	[.text]:[torch.cuda.LongTensor of size 35x32 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 35x32 (GPU 0)]
tensor([[    9,  1745,    74,  ...,     7,    24,    97],
        [   11,    12,    90,  ...,     2, 31704,  3286],
        [ 3932,     4,  1115,  ...,   151,    23,     3],
        ...,
        [   17,  2443,   631,  ...,     9,    36,    15],
        [ 3932,  6097,  4456,  ...,    11,  9466,  1527],
        [ 4429,     2,    13,  ...,    11,     2,    24]], device='cuda:0')
tensor([[   11,    12,    90,  ...,     2, 31704,  3286],
        [ 3932,     4,  1115,  ...,   151,    23,     3],
        [ 4429,    62,  5602,  ...,     3,     6,  7833],
        ...,
        [ 3932,  6097,  4456,  ...,    11,  9466,  1527],
        [ 4429,     2,    13,  ...,    11,     2,    24],
        [  852,  4316,  2402,  ...,    11,   153,   190]], device='cuda:0')


In [166]:
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_name = DatasetType.WikiText2.name
train_token_ids_loader, val_token_ids_loader, field_processor, vocabulary = get_data_loaders_causal(DATA_DIR_PATH, dataset_name, batch_size, device)

# Verify that the mask logic is correct
pad_token_id = src_field_processor.vocab.stoi[PAD_TOKEN]
for batch in train_token_ids_loader:
    # Visually inspect that masks make sense
    print(batch)
    print(batch.text)
    print(batch.target)
    # src_padding_mask, trg_mask, num_src_tokens, num_trg_tokens = get_masks_and_count_tokens(batch.src, batch.trg, pad_token_id, device)
    break

# Check vocab size
print(f'Vocabulary size={len(vocabulary)}')
# print(f'Target vocabulary size={len(field_processor.vocab)}')

# Show text from token loader
sample_text_from_loader(field_processor, field_processor, train_token_ids_loader)



downloading wikitext-2-v1.zip
wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:00<00:00, 7.43MB/s]
extracting
Time it took to prepare the iterator: 18.185972 seconds.

[torchtext.data.batch.Batch of size 32]
	[.text]:[torch.cuda.LongTensor of size 35x32 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 35x32 (GPU 0)]
tensor([[    9,  1745,    74,  ...,     7,    24,    97],
        [   11,    12,    90,  ...,     2, 31704,  3286],
        [ 3932,     4,  1115,  ...,   151,    23,     3],
        ...,
        [   17,  2443,   631,  ...,     9,    36,    15],
        [ 3932,  6097,  4456,  ...,    11,  9466,  1527],
        [ 4429,     2,    13,  ...,    11,     2,    24]], device='cuda:0')
tensor([[   11,    12,    90,  ...,     2, 31704,  3286],
        [ 3932,     4,  1115,  ...,   151,    23,     3],
        [ 4429,    62,  5602,  ...,     3,     6,  7833],
        ...,
        [ 3932,  6097,  4456,  ...,    11,  9466,  1527],
        [ 4429,     2,    13,  ...,    11,     2,    

In [86]:
from torchtext import vocab

In [ ]:
train_dataset, val_dataset, src_field_processor, trg_field_processor = get_datasets_and_vocabs(DATA_DIR_PATH, language_direction, dataset_name == DatasetType.IWSLT.name, use_caching_mechanism=False)
trained1 = train_dataset


In [ ]:
print(len(trained1.examples))

print(trained1.examples[1].text)


In [ ]:
len(trained2.examples[0].text)

In [ ]:
train_dataset.examples

In [57]:
train_dataset, val_dataset, field_processor = get_datasets_and_vocab_causal(DATA_DIR_PATH)
trained2 = train_dataset

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
Time it took to prepare the data: 2.646358 seconds.


AttributeError: 'Batch' object has no attribute 'input_fields'

In [ ]:
val_dataset

In [56]:
class DatasetType(enum.Enum):
    IWSLT = 0,
    WMT14 = 1,
    PennTreebank = 2,
    WikiText2 = 3,
    WikiText103 = 4

def get_datasets_and_vocab_causal(dataset_path, dataset_name= DatasetType.PennTreebank.name, use_caching_mechanism=False):
    # load data
    dataset_name= DatasetType.PennTreebank.name
    dataset = getattr(datasets, dataset_name) # should not be translation datsets
    spacy_en = spacy.load('en_core_web_sm')

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]

    field_processor = Field(tokenize=tokenize_en, init_token=BOS_TOKEN, eos_token=EOS_TOKEN, pad_token=PAD_TOKEN, batch_first=True)

    # fields = [('src', src_field_processor), ('trg', trg_field_processor)]
    # MAX_LEN = 100  # filter out examples that have more than MAX_LEN tokens
    # filter_pred = lambda x: len(x.src) <= MAX_LEN and len(x.trg) <= MAX_LEN

    # tokenize data
    # create datasets
    prefix = 'causal_' + dataset_name
    train_cache_path = os.path.join(dataset_path, f'{prefix}_train_cache.csv')
    val_cache_path = os.path.join(dataset_path, f'{prefix}_val_cache.csv')
    test_cache_path = os.path.join(dataset_path, f'{prefix}_test_cache.csv')

    # This simple caching mechanism gave me ~30x speedup on my machine! From ~70s -> ~2.5s!
    ts = time.time()
    if not use_caching_mechanism or not (os.path.exists(train_cache_path) and os.path.exists(val_cache_path)):
        train_dataset, val_dataset, test_dataset = dataset.splits(
            text_field=field_processor,
            root=dataset_path
        )
        train_dataset, val_dataset, test_dataset = dataset.iters()
        # save_cache(train_cache_path, train_dataset)
        # save_cache(val_cache_path, val_dataset)
        # save_cache(test_cache_path, test_dataset)
    else:
        # TODO: load from cache 
        print("did not load from cache!")
        return

    print(f'Time it took to prepare the data: {time.time() - ts:3f} seconds.')
    
    MIN_FREQ = 2
    field_processor.build_vocab(train_dataset, min_freq=MIN_FREQ)

    return train_dataset, val_dataset, field_processor


In [ ]:
field_processor

In [55]:
train_dataset

NameError: name 'train_dataset' is not defined

In [167]:
# load data
dataset_path = DATA_DIR_PATH
dataset_name= DatasetType.WikiText2.name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ts = time.time()
# prep dataset
dataset = getattr(datasets, dataset_name)  # should not be translation datsets
spacy_en = spacy.load('en')

# prep field processor (vocab)
def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

field_processor = Field(tokenize=tokenizer, init_token=BOS_TOKEN,
                    eos_token=EOS_TOKEN, pad_token=PAD_TOKEN, batch_first=True)

train, validation, test = dataset.splits(text_field=field_processor, root=dataset_path)
field_processor.build_vocab(train, validation, test, min_freq=1)


# prep iterator
train_token_ids_loader, val_token_ids_loader, test_token_ids_loader = dataset.iters(
batch_size=batch_size, root=dataset_path, device=device)

# get vocab
# vocabulary = vocab.build_vocab_from_iterator(train_token_ids_loader)
vocabulary = {}
print(f'Time it took to prepare the iterator: {time.time() - ts:3f} seconds.')


Time it took to prepare the iterator: 17.314270 seconds.


In [153]:
# train.examples[0].text
for batch in train_token_ids_loader:
    print(batch)
    print(batch)
    break
# vocabulary = vocab.build_vocab_from_iterator()
# 


[torchtext.data.batch.Batch of size 32]
	[.text]:[torch.cuda.LongTensor of size 35x32 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 35x32 (GPU 0)]


AttributeError: 'Batch' object has no attribute 'input_fields'

In [165]:
# combined = train.examples[0].text + test.examples[0].text + validation.examples[0].text
# len(catch_unique(combined))
dataset.vocab

AttributeError: type object 'PennTreebank' has no attribute 'vocab'

In [161]:
def catch_unique(list_in):
   # intilize an empty list
   unq_list = []

   # Check for elements
   for x in list_in:
      # check if exists in unq_list
      if x not in unq_list:
         unq_list.append(x)
         # print list
   return unq_list

In [173]:
import io
import torch
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

url = 'https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip'
test_filepath, valid_filepath, train_filepath = extract_archive(download_from_url(url))
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer,
                                      iter(io.open(train_filepath,
                                                   encoding="utf8"))))


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:00<00:00, 7.46MB/s]
36718lines [00:01, 25487.53lines/s]


In [175]:
len(vocab)

28783

In [195]:
woo= iter(io.open(train_filepath,encoding="utf8"))

In [196]:
trainl = list(woo)

In [188]:
dataset_path = DATA_DIR_PATH
dataset_name= DatasetType.WikiText2.name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ts = time.time()
# prep dataset
dataset = getattr(datasets, dataset_name)  # should not be translation datsets
spacy_en = spacy.load('en')

# prep field processor (vocab)
def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

field_processor = Field(tokenize=tokenizer, init_token=BOS_TOKEN,
                    eos_token=EOS_TOKEN, pad_token=PAD_TOKEN, batch_first=True)

train, validation, test = dataset.splits(text_field=field_processor, root=dataset_path)



In [198]:
len(trainl)


36718

In [199]:
train.examples[0]

In [200]:
def data_process(raw_text_iter):
  data = [torch.tensor([vocab[token] for token in tokenizer(item)],
                       dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_data = data_process(iter(io.open(train_filepath, encoding="utf8")))
val_data = data_process(iter(io.open(valid_filepath, encoding="utf8")))
test_data = data_process(iter(io.open(test_filepath, encoding="utf8")))

In [203]:
train_data

tensor([[   0, 1080,    5,  ...,  966,    4,    3],
        [   0,   11,    0,  ...,    5,    0,    0],
        [  10, 9222,    0,  ...,    2,    0,  221],
        ...,
        [   3,    2,    7,  ...,   28,    5, 4811],
        [  53,  352,  272,  ...,  780,   37,    4],
        [   7, 2410,    4,  ...,  300,  133,    0]], device='cuda:0')

In [202]:
def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)